In [67]:
import psycopg2
import getpass
import pandas as pd
import datetime
import pyodbc
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [77]:
teamproductyear_id = 2 #1,...,123

In [78]:
# connect to SQL Server.
server = '34.206.73.189' 
database = 'datascience' 
username = getpass.getpass(prompt='Enter your username') 
password = getpass.getpass(prompt='Enter your password')
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
query = f'''select teamproductyearid,lkupclientid,clientcode,productgrouping,trainseasonyear,testseasonyear,facttestyear,facttestprevyear,stlrDBName from ds.productyear_all r where teamproductyearid ={teamproductyear_id} ;'''
dfparam3 = pd.read_sql(query, cnxn)
cnxn.commit()
cursor.close()

In [79]:
dfparam3.head()

,teamproductyearid,lkupclientid,clientcode,productgrouping,trainseasonyear,testseasonyear,facttestyear,facttestprevyear,stlrDBName
0,2,9,hops,Mini Plans / Flex Plans,2019,2021,2021,None,stlrMILB


In [80]:
client_id = dfparam3._get_value(0,'lkupclientid')
client_code= dfparam3._get_value(0,'clientcode')
product_grouping =dfparam3._get_value(0,'productgrouping') 
stlrdbname= dfparam3._get_value(0,'stlrDBName') 

### select the MAX(dimDateId) for the specific lkupclientId and Product in factCustomerRetentionScore

In [81]:
# connect to SQL Server.
server = '34.206.73.189'
database = stlrdbname
username = getpass.getpass(prompt='Enter your username') 
password = getpass.getpass(prompt='Enter your password')
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor2 = cnxn.cursor()
MAXDIMDATE =  f'''select MAX(dimDateId) as maxDimDateId from dw.factCustomerretentionScore where lkupclientid ={client_id}  and product in({"'"+ str(product_grouping) + "'"}) and currVersnFlag =1;'''
dfMaxDate = pd.read_sql(MAXDIMDATE, cnxn)
cnxn.commit()
cursor2.close()
dfMaxDate

,maxDimDateId
0,20211110


### select dimCustomerMasterIds that exist in fatCustomerRetentionScore and don't exist in dimCustomerMaster table

In [86]:
# connect to SQL Server.
server = '34.206.73.189' 
database = stlrdbname
username = getpass.getpass(prompt='Enter your username') 
password = getpass.getpass(prompt='Enter your password') 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor3 = cnxn.cursor()
dimCustomerMasterIDNull =  f'''SELECT customerNumber as dimCustomerMasterId FROM dw.factCustomerretentionScore t LEFT OUTER JOIN dw.dimCustomerMaster m ON m.dimCustomerMasterId= t.customerNumber where t.lkupclientid ={client_id}  AND t.product in({"'"+ str(product_grouping) + "'"}) AND currVersnFlag =1 AND m.dimCustomerMasterId IS NULL;'''
dfCNT = pd.read_sql(dimCustomerMasterIDNull, cnxn)
cnxn.commit()
cursor3.close()
dfCNT

,dimCustomerMasterId
0,312509325


### select The number of records group by each year for one specific lkupClientId and Product

In [87]:
# connect to SQL Server.
server = '34.206.73.189' 
database = stlrdbname
username = getpass.getpass(prompt='Enter your username') 
password = getpass.getpass(prompt='Enter your password') 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor4 = cnxn.cursor()
cntPerYear =  f'''select seasonYear, count(*) cntPerYear from dw.factCustomerRetentionScore where lkupclientid ={client_id}  AND product in({"'"+ str(product_grouping) + "'"}) AND currVersnFlag =1 group by seasonYear;'''
dfcntPerYear = pd.read_sql(cntPerYear, cnxn)
cnxn.commit()
cursor4.close()
dfcntPerYear

,seasonYear,cntPerYear
0,2021,287


### select those records which the testSeasonYear in factCustomerRetentionScore table is not equal to facttestyear in ds.productyear_all table where currVersnFlag = 1 and they don't have data for previous year

In [88]:
# connect to SQL Server.
server = '34.206.73.189' 
database = stlrdbname
username = getpass.getpass(prompt='Enter your username') 
password = getpass.getpass(prompt='Enter your password')
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor4 = cnxn.cursor()
testSeasonYear =  f'''select b.*  from (select lkupClientId,product,seasonYear testSeasonYear from dw.factCustomerRetentionScore t where t.lkupclientid ={client_id}  AND product in({"'"+ str(product_grouping) + "'"}) AND currVersnFlag = 1 ) b INNER JOIN [DataScience].[DS].[productyear_all] p on b.lkupClientId=p.lkupclientid AND b.product= p.productgrouping where b.testSeasonYear <> p.facttestyear and p.isprevyearflag = 0;'''
dfTestSeasonYear = pd.read_sql(testSeasonYear, cnxn)
cnxn.commit()
cursor4.close()
dfTestSeasonYear

,lkupClientId,product,testSeasonYear
